# Roll decay equation

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import sympy as sp
from rolldecayestimators.symbols import *
import rolldecayestimators.equations as equations
import rolldecayestimators.direct_estimator as direct_estimator
import rolldecayestimators.direct_linear_estimator as direct_linear_estimator
import rolldecayestimators.direct_estimator_improved as direct_estimator_improved


from latex_helpers import pylatex_extenders
import latex_helpers
import os.path
from rigidbodysimulator.substitute_dynamic_symbols import substitute_dynamic_symbols, find_name, find_derivative_name, lambdify, find_derivatives
import dill

from sympy.physics.vector.printing import vpprint, vlatex
from IPython.display import display, Math, Latex

In [3]:
latex = vlatex(equations.roll_equation_general)
display(Math(latex))

<IPython.core.display.Math object>

No external forces:

In [4]:
latex = vlatex(equations.roll_decay_equation_general)
display(Math(latex))

<IPython.core.display.Math object>

## Linear model

In [31]:
latex = vlatex(equations.linear_stiffness_equation)
display(Math(latex))

<IPython.core.display.Math object>

In [32]:
latex = vlatex(equations.roll_decay_linear_equation)
display(Math(latex))

<IPython.core.display.Math object>

In [33]:
equations.roll_decay_linear_equation

Eq(B*Derivative(phi(t), t) + GM*g*m*phi(t) + I*Derivative(phi(t), (t, 2)), 0)

In [34]:
omega_equation = sp.Eq(omega0**2,
     GM*g*m/I)
omega_equation

Eq(omega0**2, GM*g*m/I)

In [35]:
sp.solve(omega_equation,omega0**2)[0]

GM*g*m/I

In [39]:
(equations.roll_decay_linear_equation.lhs/I).expand()

B*Derivative(phi(t), t)/I + GM*g*m*phi(t)/I + Derivative(phi(t), (t, 2))

In [40]:
(equations.roll_decay_linear_equation.lhs/I).expand().subs(
    sp.solve(omega_equation,omega0**2)[0],
    omega0**2)

B*Derivative(phi(t), t)/I + omega0**2*phi(t) + Derivative(phi(t), (t, 2))

In [41]:
equations.linear_damping_equation

Eq(B/I, zeta)

In [5]:
latex = vlatex(direct_linear_estimator.roll_diff_equation)
display(Math(latex))

<IPython.core.display.Math object>

In [7]:
## Save to tex
conatiner = pylatex_extenders.GeneralContainer()
math = pylatex_extenders.Equation(direct_linear_estimator.roll_diff_equation,label='eq:rollDiffEquationLinear')
conatiner.append(math)
file_path = os.path.join(latex_helpers.general_content_path,'roll_diff_equation_linear')
conatiner.generate_tex(file_path)

In [9]:
result = sp.solve(direct_linear_estimator.roll_diff_equation, phi, 2)[0][phi]

In [10]:
equation2 = sp.Eq(lhs=phi, rhs=result)
latex = vlatex(equation2.expand())
display(Math(latex))

<IPython.core.display.Math object>

In [11]:
equations.A_equation

Eq(A, -2*zeta/omega0)

In [12]:
equations.B_equation

Eq(B, -1/omega0**2)

In [13]:
subs= {equations.A_equation.rhs:equations.A_equation.lhs,
       equations.B_equation.rhs:equations.B_equation.lhs,
    }
latex = vlatex(equation2.expand().subs(subs))
display(Math(latex))

<IPython.core.display.Math object>

In [14]:
equations.zeta_equation

Eq(zeta, -A*omega0/2)

In [15]:
equations.omega_equation

Eq(omega0, -sqrt(-1/B))

## Quadratic model

In [16]:
latex = vlatex(direct_estimator.roll_diff_equation)
display(Math(latex))

<IPython.core.display.Math object>

In [17]:
equations.C_equation

Eq(C, B*d)

In [19]:
subs= [(equations.A_equation.rhs,equations.A_equation.lhs),
       (equations.B_equation.rhs,equations.B_equation.lhs),
       (equations.C_equation.rhs,equations.C_equation.lhs),
      ]
latex = vlatex(direct_estimator.roll_diff_equation.expand().subs(subs))
display(Math(latex))

<IPython.core.display.Math object>

In [20]:
d_equation = sp.Eq(lhs=d, rhs=sp.solve(equations.C_equation,d)[0])
d_equation

Eq(d, C/B)

In [21]:
equations.velocity_equation_linear

Eq(Derivative(phi(t), t), p_old(t))

In [22]:
equations.acceleration_equation_linear

Eq(Derivative(phi(t), (t, 2)), -omega0*(omega0*phi_old(t) + 2*zeta*p_old(t)))

In [23]:
## Save to tex
conatiner = pylatex_extenders.GeneralContainer()
math = pylatex_extenders.Equation(direct_estimator.roll_diff_equation,label='eq:rollDiffEquationQuadratic')
conatiner.append(math)
file_path = os.path.join(latex_helpers.general_content_path,'roll_diff_equation_quadratic')
conatiner.generate_tex(file_path)

## Example: Analytically solve simpler equation

In [27]:
t = sp.Symbol('t')
y = me.dynamicsymbols('y')
y


y(t)

In [28]:
eq = sp.Eq(lhs=y.diff().diff(), rhs=-9.81)
eq

Eq(Derivative(y(t), (t, 2)), -9.81)

In [29]:
sp.dsolve(eq)

Eq(y(t), C1 + C2*t - 981*t**2/200)

In [30]:
eq2 = sp.Eq(lhs=y.diff().diff(), rhs=-9.81 + 0.1*y.diff())
sp.dsolve(eq2)

Eq(y(t), C1 + C2*exp(0.1*t) + 98.1*t)